# 数据预处理

创建一个人工数据集，并存储在CSV（逗号分隔值）文件

In [1]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')
    f.write('NA,Pave,127500\n')
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

从创建的CSV文件中加载原始数据集

In [2]:
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


为了处理缺失的数据，典型的方法包括*插值法*和*删除法*，
这里，我们将考虑插值法

In [10]:
inputs = data.iloc[:, 0:2]
inputs

,NumRooms,Alley
0,NaN,Pave
1,2.0,NaN
2,4.0,NaN
3,NaN,NaN


In [30]:
inputs.select_dtypes(include='number').mean()

NumRooms    3.0
dtype: float64

In [22]:
inputs.mean(numeric_only=True)

NumRooms    3.0
dtype: float64

In [27]:
inputs = inputs.fillna(inputs.mean(numeric_only=True))
inputs

,NumRooms,Alley
0,3.0,Pave
1,2.0,NaN
2,4.0,NaN
3,3.0,NaN


In [32]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = inputs.fillna(inputs.mean(numeric_only=True))
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


对于`inputs`中的类别值或离散值，我们将“NaN”视为一个类别

In [34]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0        True      False
1       2.0       False       True
2       4.0       False       True
3       3.0       False       True


现在`inputs`和`outputs`中的所有条目都是数值类型，它们可以转换为张量格式

In [35]:
inputs.to_numpy()

array([[3.0, True, False],
       [2.0, False, True],
       [4.0, False, True],
       [3.0, False, True]], dtype=object)

In [36]:
inputs.to_numpy(dtype=int)

array([[3, 1, 0],
       [2, 0, 1],
       [4, 0, 1],
       [3, 0, 1]])

In [37]:
inputs.to_numpy(dtype=float)

array([[3., 1., 0.],
       [2., 0., 1.],
       [4., 0., 1.],
       [3., 0., 1.]])

In [38]:
import torch

X = torch.tensor(inputs.to_numpy(dtype=float))
y = torch.tensor(outputs.to_numpy(dtype=float))
X, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500., 106000., 178100., 140000.], dtype=torch.float64))

## 练习

In [39]:
data

,NumRooms,Alley,Price
0,NaN,Pave,127500
1,2.0,NaN,106000
2,4.0,NaN,178100
3,NaN,NaN,140000


In [63]:
na_count = data.isna().sum().to_dict()
na_count

{'NumRooms': 2, 'Alley': 3, 'Price': 0}

In [64]:
max(na_count, key=na_count.get)

'Alley'

In [66]:
na_count = data.isna().sum()
na_count

NumRooms    2
Alley       3
Price       0
dtype: int64

In [69]:
na_count.max()

3

In [72]:
na_count.idxmax()

'Alley'

In [79]:
data.drop(columns = data.isna().sum().idxmax())

,NumRooms,Price
0,NaN,127500
1,2.0,106000
2,4.0,178100
3,NaN,140000
